In [52]:
import time
import requests
import pandas as pd
import matplotlib.pyplot as plt
from lxml import etree

ModuleNotFoundError: No module named 'matplotlib'

In [39]:
api_key = "577df7196ceeb3a853b9aac5b7bfe5640209"  # It's recommended to include your NCBI API key
email = "nawar82@gmail.com"

def search_pubmed(query):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query,
        "retmax": "10000",
        "usehistory": "y",
        "email": email,
        "api_key": api_key,
        "retmode": "json"  # Set retmode to json
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = requests.get(base_url, params=params).json()
        #id_list = data["esearchresult"]["idlist"]
        if data['esearchresult']['count'] == data['esearchresult']['retmax']:
            print(f"You have a total of {data['esearchresult']['count']} articles and getting {data['esearchresult']['retmax']} articles to fetch their abstracts")
        else:
            print(f"Please be aware that your search terms returned {data['esearchresult']['count']} articles but you are getting only {data['esearchresult']['retmax']} for fetching their abstracts")
        return data["esearchresult"]["idlist"]
    else:
        print("Error occurred while searching")
        return []

In [40]:
def fetch_abstracts(id_list):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    ids = ','.join(id_list)
    params = {
        "db": "pubmed",
        "retmode": "xml",
        "id": ids,
        "api_key": api_key
    }
    response = requests.get(base_url, params=params)
    abstracts = []
    if response.status_code == 200:
        root = etree.fromstring(response.content)
        articles = root.xpath('//PubmedArticle')
        for article in articles:
            pmid = article.find('.//PMID').text
            abstract_text_element = article.find('.//Abstract/AbstractText')
            language = article.find('.//Language').text
            journal = article.find('.//Journal/Title').text
            # Ensure abstract_text_element is not None before accessing its text attribute
            if abstract_text_element is not None and language == 'eng':
                abstract_text = get_all_text(abstract_text_element)
                # Further ensure the text is not None and not just whitespace
                if abstract_text and abstract_text.strip():
                    abstracts.append({"PMID": pmid, "Abstract": abstract_text, "Journal":journal})
    else:
        print("Error occurred while fetching details")
    return abstracts

In [41]:

query = "tsh AND Mendelian randomization"
id_list = search_pubmed(query)
abstracts = fetch_abstracts(id_list)

You have a total of 69 articles and getting 69 articles to fetch their abstracts


In [54]:


id_list = search_pubmed(query)
all_abstracts = fetch_all_abstracts(id_list, nr_of_requests)
all_abstracts


You have a total of 69 articles and getting 69 articles to fetch their abstracts


[{'PMID': '38472621',
  'Abstract': "Some observational studies have suggested the association between thyroid function and polycystic ovary syndrome (PCOS). However, it remains to be determined whether these associations are causal or not. The aim of this study was to investigate the underlying causal association between different thyroid function status and PCOS.Bidirectional Mendelian randomization (MR) analysis was conducted to explore the impact of different thyroid function statuses on PCOS. The study included 10,074 individuals with PCOS and 103,164 controls for the primary analysis, with validation analysis repeated in the FinnGen R9 and EstBB PCOS cohorts. Female-specific thyroid function GWAS data were obtained from European population, including Hyperthyroidism (22,383 cases and 54,288 controls) and Hypothyroidism (27,383 cases and 54,288 controls) from the UK Biobank, and TSH (54,288 cases and 72,167 controls) and FT4 (49,269 cases and 72,167 controls) within the reference 

In [44]:
 # Convert the list of abstracts into a pandas DataFrame
data = pd.DataFrame(all_abstracts)
data

,PMID,Abstract,Journal
0,38472621,Some observational studies have suggested the ...,Endocrine
1,38405140,Previous studies have suggested a potential as...,Frontiers in endocrinology
2,38392839,The association between thyroid function and v...,"Pathogens (Basel, Switzerland)"
3,38375193,To determine whether there is a causal relatio...,Frontiers in endocrinology
4,38368359,The role of thyroid health in temporomandibula...,BMC oral health
...,...,...,...
63,30248900,The thyroid plays a key role in development an...,Journal of clinical medicine
64,30016786,Increasing evidence suggests an association be...,Kidney & blood pressure research
65,29544020,"With population aging, prevalence of low bone ...",Journal of bone and mineral research : the off...
66,28819171,To clarify the role of thyroid function in isc...,Scientific reports


In [45]:
#Number of records grouped by the journal
cnt = data.groupby(['Journal']).size()
cnt

Journal
American journal of ophthalmology                                                                                                       1
Atherosclerosis                                                                                                                         1
BMC medicine                                                                                                                            1
BMC oral health                                                                                                                         1
BMC pulmonary medicine                                                                                                                  1
Borderline personality disorder and emotion dysregulation                                                                               1
Chinese medical journal                                                                                                                 1
Clinical endocrinology    

In [46]:
#Convert count to df
df = cnt.to_frame().reset_index()
df

,Journal,0
0,American journal of ophthalmology,1
1,Atherosclerosis,1
2,BMC medicine,1
3,BMC oral health,1
4,BMC pulmonary medicine,1
5,Borderline personality disorder and emotion dy...,1
6,Chinese medical journal,1
7,Clinical endocrinology,1
8,Computational and mathematical methods in medi...,1
9,Endocrine,1


In [47]:
df = df.rename(columns={0:'Total_articles'})
df

,Journal,Total_articles
0,American journal of ophthalmology,1
1,Atherosclerosis,1
2,BMC medicine,1
3,BMC oral health,1
4,BMC pulmonary medicine,1
5,Borderline personality disorder and emotion dy...,1
6,Chinese medical journal,1
7,Clinical endocrinology,1
8,Computational and mathematical methods in medi...,1
9,Endocrine,1


In [48]:
df = df.sort_values(by='Total_articles', ascending=False)
df

,Journal,Total_articles
34,The Journal of clinical endocrinology and meta...,14
14,Frontiers in endocrinology,9
35,Thyroid : official journal of the American Thy...,7
11,Endocrinology,2
16,Frontiers in immunology,2
12,European journal of epidemiology,2
21,Journal of affective disorders,2
30,Nature communications,2
29,Kidney & blood pressure research,1
28,Journal of personalized medicine,1


In [60]:

top_n= df.nlargest(10,'Total_articles')
print(top_n)
#return top_n

                                              Journal  Total_articles
34  The Journal of clinical endocrinology and meta...              14
14                         Frontiers in endocrinology               9
35  Thyroid : official journal of the American Thy...               7
11                                      Endocrinology               2
16                            Frontiers in immunology               2
12                   European journal of epidemiology               2
21                     Journal of affective disorders               2
30                              Nature communications               2
29                   Kidney & blood pressure research               1
28                   Journal of personalized medicine               1


In [ ]:
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.bar(top_n['Journal'], top_n['Total_articles'], color='skyblue')
plt.xlabel('Journal')
plt.ylabel('Number of Articles')
plt.title('Top Journals by Number of Articles')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#return top_n